# Final merge of the road and bridge datsets

In [1]:
import pandas as pd
import numpy as np
from haversine import haversine, Unit

In [2]:
documented_intersections_start_end = pd.read_csv('../data/semi-processed/identified_roads_final.csv')
roads_in_simulation = documented_intersections_start_end.road.unique().tolist()
len(roads_in_simulation)

26

In [3]:
documented_intersections_start_end

Unnamed: 0   road  chainage  LRPName        lat        lon  gap  \
0            0     N1     0.000     LRPS  23.706028  90.443333  NaN   
1            1     N1     8.763  LRP009a  23.706083  90.521527  NaN   
2            2     N1    45.322  LRP046a  23.530749  90.781610  NaN   
3            3     N1    47.224  LRP047a  23.530694  90.792444  NaN   
4            4     N1    55.477  LRP056c  23.512667  90.871416  NaN   
..         ...    ...       ...      ...        ...        ...  ...   
85          87  Z1048    31.130     LRPE  23.149722  91.325194  NaN   
86          66  Z1402     0.000     LRPS  23.259389  90.696500  NaN   
87          88  Z1402    43.849     LRPE  23.530749  90.781610  NaN   
88          89  Z2013     0.000     LRPS  24.867305  91.856806  NaN   
89          67  Z2013    25.045     LRPE  24.657222  91.828222  NaN   

              type                                               name  \
0           Others  Start of Road  after Jatrabari Flyover infront...   
1    sideroad,left                                Road to Sylhet (N2)   
2   sideroad,right                              Z1402, road to Pennai   
3   sideroad,right                                              Z1044   
4    sideroad,left                        Z1042, road to Bancharampur   
..             ...                                                ...   
85          Others       End of road at Chewara Intersection with N1.   
86          Others                            Start of road from R140   
87          Others                             Meet with N1 at Pennai   
88          Others            Road Start From N2 at Chandipool Sylhet   
89          Others                      Road End With Borobaga River.   

         model_type road_to  distance_to_closest  closest_lat  closest_lon  \
0        sourcesink     NaN                  NaN          NaN          NaN   
1      intersection      N2            20.374160    23.705917    90.521444   
2      intersection   Z1402             0.000000    23.530749    90.781610   
3      intersection   Z1044             0.000000    23.530694    90.792444   
4      intersection   Z1042             3.091223    23.512639    90.871416   
..              ...     ...                  ...          ...          ...   
85  intersourcesink      N1            31.387443    23.149694    91.325499   
86       sourcesink     NaN                  NaN          NaN          NaN   
87  intersourcesink      N1             0.000000    23.530749    90.781610   
88  intersourcesink      N2             0.000000    24.867305    91.856806   
89       sourcesink     NaN                  NaN          NaN          NaN   

   closest_lrp                         closest_name    closest_type  \
0          NaN                                  NaN             NaN   
1         LRPS       Road Start from N1 at Katchpur          Others   
2         LRPE               Meet with N1 at Pennai          Others   
3         LRPS    Start of Road From N1 at Gouripur          Others   
4         LRPS  Start of Road from N1 at Eliotganj.          Others   
..         ...                                  ...             ...   
85     LRP130a            Road to Nangolhat (Z1048)  sideroad,right   
86         NaN                                  NaN             NaN   
87     LRP046a                Z1402, road to Pennai  sideroad,right   
88     LRP225b             Road to Balaganj (Z2013)  sideroad,right   
89         NaN                                  NaN             NaN   

    closest_chainage  intersection  
0                NaN           NaN  
1              0.000           NaN  
2             43.849           NaN  
3              0.000           NaN  
4              0.000           NaN  
..               ...           ...  
85           128.009  intersection  
86               NaN           NaN  
87            45.322  intersection  
88           225.373  intersection  
89               NaN           NaN  

[90 rows x 19 columns]

In [4]:
# documented_intersections_start_end[['model_type','road_to','road']]

In [5]:
# documented_intersections_start_end.model_type.value_counts()

# Import the bridges

In [6]:
bridges = pd.read_csv('../data/semi-processed/bridges_vul_index.csv')
bridges['model_type']='bridge'
bridges.drop(columns=['Unnamed: 0'], inplace=True)

# This is to filter out the cleaning we did at the distance to closest< threshold which was not done with the list used for the bridges
bridges = bridges[bridges['road'].isin(roads_in_simulation)]
bridges

Unnamed: 0.1   road      km                   type  LRPName  \
0                0  Z1044  39.882            Box Culvert  LRP040b   
1                1  Z1044  39.444            Box Culvert  LRP040a   
2                2  Z1044  38.698       PC Girder Bridge  LRP039c   
3                3  Z1044  38.530            Box Culvert  LRP039b   
4                4  Z1044  38.271            Box Culvert  LRP039a   
...            ...    ...     ...                    ...      ...   
2380          2380   R240  47.812            Box Culvert  LRP047e   
2381          2381   R240  48.610            Box Culvert  LRP048c   
2382          2382   R240  48.270  Baily with Steel Deck  LRP048a   
2383          2383   R240  49.160            Box Culvert  LRP049a   
2384          2384   R240  49.333            Box Culvert  LRP049c   

                               name  length condition  structureNr  \
0              HOZIGANG BOX CULVERT   12.30         A       102609   
1                 BODORPUR SUBMARGE    1.50         A       102607   
2                   BODORPUR BRIDGE   64.60         A       102608   
3          kathalia sub box culvert    1.50         A       102604   
4                  KATHALIA CULVERT    3.00         A       102602   
...                             ...     ...       ...          ...   
2380             KHURSI BOX CULVERT   12.23         B       102244   
2381             RAIPUR BOX CULVERT    1.50         A       102249   
2382  PAIPUR BAILEY WITH STEEL DECK   24.30         C       102247   
2383           AUSKANDI BOX CULVERT   12.30         B       102250   
2384           AUSKAMDI BOX CULVERT    1.50         A       102253   

                                               roadName  ...     zone  \
0                         Gouripur-Kachua-Hajiganj Road  ...  Comilla   
1                         Gouripur-Kachua-Hajiganj Road  ...  Comilla   
2                         Gouripur-Kachua-Hajiganj Road  ...  Comilla   
3                         Gouripur-Kachua-Hajiganj Road  ...  Comilla   
4                         Gouripur-Kachua-Hajiganj Road  ...  Comilla   
...                                                 ...  ...      ...   
2380  Shaistaganj-Habiganj-Nabiganj-Sherpur (Auskand...  ...   Sylhet   
2381  Shaistaganj-Habiganj-Nabiganj-Sherpur (Auskand...  ...   Sylhet   
2382  Shaistaganj-Habiganj-Nabiganj-Sherpur (Auskand...  ...   Sylhet   
2383  Shaistaganj-Habiganj-Nabiganj-Sherpur (Auskand...  ...   Sylhet   
2384  Shaistaganj-Habiganj-Nabiganj-Sherpur (Auskand...  ...   Sylhet   

             circle  division  sub-division        lat        lon  \
0           Comilla  Chandpur      Hajigonj  23.254886  90.849722   
1           Comilla  Chandpur      Hajigonj  23.257385  90.853146   
2           Comilla  Chandpur      Hajigonj  23.261470  90.858600   
3           Comilla  Chandpur      Hajigonj  23.262256  90.859989   
4           Comilla  Chandpur      Hajigonj  23.263467  90.862129   
...             ...       ...           ...        ...        ...   
2380  Moulavi Bazar  Habiganj      Habigonj  24.590565  91.585901   
2381  Moulavi Bazar  Habiganj      Habigonj  24.590976  91.592102   
2382  Moulavi Bazar  Habiganj      Habigonj  24.590992  91.590435   
2383  Moulavi Bazar  Habiganj      Habigonj  24.591276  91.599333   
2384  Moulavi Bazar  Habiganj      Habigonj  24.591425  91.601015   

     EstimatedLoc lanes  vul_cat  model_type  
0     interpolate   2.0      2.0      bridge  
1     interpolate   2.0      2.0      bridge  
2     interpolate   2.0      2.0      bridge  
3     interpolate   2.0      2.0      bridge  
4     interpolate   2.0      2.0      bridge  
...           ...   ...      ...         ...  
2380  interpolate   2.0      2.0      bridge  
2381  interpolate   2.0      2.0      bridge  
2382  interpolate   2.0      2.0      bridge  
2383  interpolate   2.0      2.0      bridge  
2384  interpolate   2.0      2.0      bridge  

[2385 rows x 24 columns]

In [7]:
data_points = pd.concat([bridges, documented_intersections_start_end], ignore_index=True)
data_points.sort_values(by=['road','chainage'], inplace=True)
data_points.drop(['Unnamed: 0.1', 'structureNr', 'width',
       'constructionYear', 'spans', 'zone', 'circle', 'division',
       'sub-division','EstimatedLoc', 'Unnamed: 0', 'gap'], axis=1, inplace=True)
data_points

road      km              type  LRPName  \
2385     N1     NaN            Others     LRPS   
2318     N1   1.800       Box Culvert  LRP001a   
2316     N1   4.925       Box Culvert  LRP004b   
2386     N1     NaN     sideroad,left  LRP009a   
2319     N1   8.976  PC Girder Bridge  LRP008b   
...     ...     ...               ...      ...   
1443  Z2013  17.796       Box Culvert  LRP017c   
1442  Z2013  18.148       Box Culvert  LRP018a   
1441  Z2013  18.732       Box Culvert  LRP018b   
1440  Z2013  18.995       Box Culvert  LRP018c   
2474  Z2013     NaN            Others     LRPE   

                                                   name  length condition  \
2385  Start of Road  after Jatrabari Flyover infront...     NaN       NaN   
2318                                                  .    11.3         A   
2316                                                  .     6.6         A   
2386                                Road to Sylhet (N2)     NaN       NaN   
2319                          KANCHPUR PC GIRDER BRIDGE   397.0         C   
...                                                 ...     ...       ...   
1443                                AZIZPUR BOX CULVERT     6.0         A   
1442                                           AZIZADUR     6.0         A   
1441                                    LONGIRA CULVERT     3.0         A   
1440                                    LONGIRA CULVERT     6.0         A   
2474                      Road End With Borobaga River.     NaN       NaN   

                                               roadName  chainage        lat  \
2385                                                NaN     0.000  23.706028   
2318  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...     1.800  23.698739   
2316  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...     4.925  23.694664   
2386                                                NaN     8.763  23.706083   
2319  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...     8.976  23.705060   
...                                                 ...       ...        ...   
1443          Sylhet (Telikhal)-Sultanpur-Balaganj Road    17.796  24.714799   
1442          Sylhet (Telikhal)-Sultanpur-Balaganj Road    18.148  24.711685   
1441          Sylhet (Telikhal)-Sultanpur-Balaganj Road    18.732  24.706707   
1440          Sylhet (Telikhal)-Sultanpur-Balaganj Road    18.995  24.704475   
2474                                                NaN    25.045  24.657222   

      ...    model_type  road_to  distance_to_closest closest_lat closest_lon  \
2385  ...    sourcesink      NaN                  NaN         NaN         NaN   
2318  ...        bridge      NaN                  NaN         NaN         NaN   
2316  ...        bridge      NaN                  NaN         NaN         NaN   
2386  ...  intersection       N2             20.37416   23.705917   90.521444   
2319  ...        bridge      NaN                  NaN         NaN         NaN   
...   ...           ...      ...                  ...         ...         ...   
1443  ...        bridge      NaN                  NaN         NaN         NaN   
1442  ...        bridge      NaN                  NaN         NaN         NaN   
1441  ...        bridge      NaN                  NaN         NaN         NaN   
1440  ...        bridge      NaN                  NaN         NaN         NaN   
2474  ...    sourcesink      NaN                  NaN         NaN         NaN   

      closest_lrp                    closest_name  closest_type  \
2385          NaN                             NaN           NaN   
2318          NaN                             NaN           NaN   
2316          NaN                             NaN           NaN   
2386         LRPS  Road Start from N1 at Katchpur        Others   
2319          NaN                             NaN           NaN   
...           ...                             ...           ...   
1443          NaN                             NaN           NaN   
1442       

In [8]:
# Make sure the intersections and (inter)sourcesinks have a length of 0
for i,row in data_points.iterrows():
    if row['model_type']!='bridge':
        data_points.at[i,'length']=0
        
# data_points.length.value_counts()

## Now, lets add the inbetween links and calculate the length of the links

In [9]:
def add_links_rows(df):
    nans = np.where(np.empty_like(df.values), np.nan, np.nan)    
    data = np.hstack([df.values, nans]).reshape(-1, df.shape[1])
    new_df = pd.DataFrame(data, columns=df.columns)
    return new_df.iloc[:-1, :]

In [10]:
N1N2_empty_links = add_links_rows(data_points)
N1N2_empty_links

road      km         type  LRPName  \
0        N1     NaN       Others     LRPS   
1       NaN     NaN          NaN      NaN   
2        N1     1.8  Box Culvert  LRP001a   
3       NaN     NaN          NaN      NaN   
4        N1   4.925  Box Culvert  LRP004b   
...     ...     ...          ...      ...   
4944  Z2013  18.732  Box Culvert  LRP018b   
4945    NaN     NaN          NaN      NaN   
4946  Z2013  18.995  Box Culvert  LRP018c   
4947    NaN     NaN          NaN      NaN   
4948  Z2013     NaN       Others     LRPE   

                                                   name length condition  \
0     Start of Road  after Jatrabari Flyover infront...    0.0       NaN   
1                                                   NaN    NaN       NaN   
2                                                     .   11.3         A   
3                                                   NaN    NaN       NaN   
4                                                     .    6.6         A   
...                                                 ...    ...       ...   
4944                                    LONGIRA CULVERT    3.0         A   
4945                                                NaN    NaN       NaN   
4946                                    LONGIRA CULVERT    6.0         A   
4947                                                NaN    NaN       NaN   
4948                      Road End With Borobaga River.    0.0       NaN   

                                               roadName chainage        lat  \
0                                                   NaN      0.0  23.706028   
1                                                   NaN      NaN        NaN   
2     Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...      1.8  23.698739   
3                                                   NaN      NaN        NaN   
4     Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...    4.925  23.694664   
...                                                 ...      ...        ...   
4944          Sylhet (Telikhal)-Sultanpur-Balaganj Road   18.732  24.706707   
4945                                                NaN      NaN        NaN   
4946          Sylhet (Telikhal)-Sultanpur-Balaganj Road   18.995  24.704475   
4947                                                NaN      NaN        NaN   
4948                                                NaN   25.045  24.657222   

      ...  model_type road_to distance_to_closest closest_lat closest_lon  \
0     ...  sourcesink     NaN                 NaN         NaN         NaN   
1     ...         NaN     NaN                 NaN         NaN         NaN   
2     ...      bridge     NaN                 NaN         NaN         NaN   
3     ...         NaN     NaN                 NaN         NaN         NaN   
4     ...      bridge     NaN                 NaN         NaN         NaN   
...   ...         ...     ...                 ...         ...         ...   
4944  ...      bridge     NaN                 NaN         NaN         NaN   
4945  ...         NaN     NaN                 NaN         NaN         NaN   
4946  ...      bridge     NaN                 NaN         NaN         NaN   
4947  ...         NaN     NaN                 NaN         NaN         NaN   
4948  ...  sourcesink     NaN                 NaN         NaN         NaN   

     closest_lrp closest_name closest_type closest_chainage intersection  
0            NaN          NaN          NaN              NaN          NaN  
1            NaN          NaN          NaN              NaN          NaN  
2            NaN          NaN          NaN              NaN          NaN  
3            NaN          NaN          NaN              NaN          NaN  
4            NaN          NaN          NaN              NaN          NaN  
...          ...          ...          ...              ...          ...  
4944         NaN          NaN          NaN              NaN          NaN  
4945         NaN          NaN          NaN              NaN          NaN  
4946

# Fill in the links

nshsh

In [11]:
def fill_links(df):
    countlink = 1
    countlrp = 1
    ab = 0
    df['road'].bfill( inplace=True)
    for index in range(len(df)):
        
        ## FILLING THE LINKS
        if pd.isna(df.loc[index,'LRPName']):
            
            df.at[index, 'length'] = haversine((df.at[(index-1), 'lat'], df.at[(index-1), 'lon']), (df.at[(index+1), 'lat'], df.at[(index+1), 'lon']), unit=Unit.METERS)
            
            df.at[index, 'lat'] = df.at[(index+1), 'lat']+ (df.at[(index+1), 'lat']-df.at[(index-1), 'lat'])/2
            df.at[index, 'lon'] = df.at[(index+1), 'lon']+ (df.at[(index+1), 'lon']-df.at[(index-1), 'lon'])/2
            df.at[index, 'model_type'] ='link'
            
            ## NAMING THE LINKS INBETWEEN
            if index + 2< len(df):
                ## IF YOUR PREVIOUS IS A SOURCE, OR YOUR NEXT IS A SOURCE, THEN 
                if ((df.at[index-1, 'model_type'] =='sourcesink') and (df.at[index+1, 'model_type'] =='sourcesink')):
                    ## IF YOU HAVE A NEGATIVE LENGTH, THEN YOU ARE INBETWEEN TWO DIFFERENT ROADS. SO YOU DONT EXIST!! THATS WHY MODEL TYPE GOES TO "not a real link"
                    df.at[index, 'name'] ='not a real link'
                    countlink = 1
                
                # ## IF YOUR PREVIOUS AND NEXT IS A SOURCE, HIDDEN DUE TO DUPLICATES (INTERSECTION& SOURCESINK)   
                elif (df.at[index-1, 'model_type']=='intersourcesink') and (df.at[index+1, 'model_type']=='intersourcesink'): 
                     df.at[index, 'name'] ='not a real link'
                     countlink = 1
                     
                #IF YOU ARE INBETWEEN SS-ISS OR ISS-SS     
                elif ((df.at[index-1, 'model_type']=='intersourcesink') and (df.at[index+1, 'model_type']=='sourcesink')) or  ((df.at[index-1, 'model_type']=='sourcesink') and (df.at[index+1, 'model_type']=='intersourcesink')):
                    df.at[index, 'name'] ='not a real link'
                    countlink = 1
                
                    
                ## IF THE ROAD OF THE NEXT LINK (2 indexes away) IS THE SAME AS YOUR ROAD, AND YOU HAVE A POSITIVE LINK
                ##THEN NAME YOURSELF THE NEXT LINK-NAME AVAILABLE NAME
                elif df.loc[(index+2),'road']==df.loc[index,'road']:
                    df.at[index, 'name'] ='link'+' '+str(countlink)
                    countlink +=1
                ## IF THE NEXT LINK IS ON ANOTHER ROAD THEN NAME YOURSELF AND RESET THE COUNTER SO THE NEXT LINK WILL BE "link 1" OF HIS OWN ROAD
                elif df.loc[(index+2),'road']!=df.loc[index,'road']:
                    df.at[index, 'name'] ='link'+' '+str(countlink)
                    countlink = 1
        
        elif index + 2< len(df):
            if (df.loc[index,'model_type']=='model_type'):
                ab+=1
                df.at[index, 'length'] = haversine(
                    (df.at[(index-1), 'lat'], df.at[(index-1), 'lon']), 
                    (df.at[(index+1), 'lat'], df.at[(index+1), 'lon']), unit=Unit.METERS)
            
                    
        ## FILLING THE BRIGDES        
        else:
            # Naming LRPS in the dataset
            if index + 2< len(df):
                ## If the road of the next LRP is still the same and the bridge length is positive, name yourself the next available name and counter+1
                if (df.loc[(index+2),'road']==df.loc[index,'road']):
                    name ='bridge'+' '+str(countlrp)
                    countlrp +=1
                ## If the next LRP is on a different road, then name yourself and reset the counter after
                elif (df.loc[(index+2),'road']!=df.loc[index,'road']):
                    name ='bridge'+' '+str(countlrp)
                    countlrp = 1
            # 
            if df.at[index,'name']=='.' or df.at[index, 'name']==0:
                df.at[index, 'name']= name
        
    return df

In [12]:
N1N2_filled_links = fill_links(N1N2_empty_links)
N1N2_filled_links[1:]

road      km         type  LRPName                           name  \
1        N1     NaN          NaN      NaN                         link 1   
2        N1     1.8  Box Culvert  LRP001a                              .   
3        N1     NaN          NaN      NaN                         link 2   
4        N1   4.925  Box Culvert  LRP004b                              .   
5        N1     NaN          NaN      NaN                         link 3   
...     ...     ...          ...      ...                            ...   
4944  Z2013  18.732  Box Culvert  LRP018b                LONGIRA CULVERT   
4945  Z2013     NaN          NaN      NaN                        link 23   
4946  Z2013  18.995  Box Culvert  LRP018c                LONGIRA CULVERT   
4947  Z2013     NaN          NaN      NaN                            NaN   
4948  Z2013     NaN       Others     LRPE  Road End With Borobaga River.   

           length condition  \
1     1776.650373       NaN   
2            11.3         A   
3     2978.663177       NaN   
4             6.6         A   
5     3663.629718       NaN   
...           ...       ...   
4944          3.0         A   
4945    251.56718       NaN   
4946          6.0         A   
4947  5331.997408       NaN   
4948          0.0       NaN   

                                               roadName chainage        lat  \
1                                                   NaN      NaN  23.695094   
2     Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...      1.8  23.698739   
3                                                   NaN      NaN  23.692626   
4     Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...    4.925  23.694664   
5                                                   NaN      NaN  23.711793   
...                                                 ...      ...        ...   
4944          Sylhet (Telikhal)-Sultanpur-Balaganj Road   18.732  24.706707   
4945                                                NaN      NaN  24.703359   
4946          Sylhet (Telikhal)-Sultanpur-Balaganj Road   18.995  24.704475   
4947                                                NaN      NaN  24.633596   
4948                                                NaN   25.045  24.657222   

      ...  model_type road_to distance_to_closest closest_lat closest_lon  \
1     ...        link     NaN                 NaN         NaN         NaN   
2     ...      bridge     NaN                 NaN         NaN         NaN   
3     ...        link     NaN                 NaN         NaN         NaN   
4     ...      bridge     NaN                 NaN         NaN         NaN   
5     ...        link     NaN                 NaN         NaN         NaN   
...   ...         ...     ...                 ...         ...         ...   
4944  ...      bridge     NaN                 NaN         NaN         NaN   
4945  ...        link     NaN                 NaN         NaN         NaN   
4946  ...      bridge     NaN                 NaN         NaN         NaN   
4947  ...        link     NaN                 NaN         NaN         NaN   
4948  ...  sourcesink     NaN                 NaN         NaN         NaN   

     closest_lrp closest_name closest_type closest_chainage intersection  
1            NaN          NaN          NaN              NaN          NaN  
2            NaN          NaN          NaN              NaN          NaN  
3            NaN          NaN          NaN              NaN          NaN  
4            NaN          NaN          NaN              NaN          NaN  
5            NaN          NaN          NaN              NaN          NaN  
...          ...          ...          ...              ...          ...  
4944         NaN          NaN          NaN              NaN          NaN  
4945         NaN          NaN          NaN              NaN          NaN  
4946         NaN          NaN          NaN              NaN          NaN  
4947         NaN          NaN          NaN              NaN          NaN  
4948         NaN      

In [13]:
fakes = N1N2_filled_links[N1N2_filled_links['name']=='not a real link']
fakes

road   km type LRPName             name         length condition  \
1289   N102  NaN  NaN     NaN  not a real link  315239.756549       NaN   
1473   N104  NaN  NaN     NaN  not a real link  119332.643753       NaN   
1559   N105  NaN  NaN     NaN  not a real link  111610.249491       NaN   
1655     N2  NaN  NaN     NaN  not a real link   35638.213037       NaN   
2617   N204  NaN  NaN     NaN  not a real link  131110.386377       NaN   
2685   N207  NaN  NaN     NaN  not a real link    4538.255954       NaN   
2823   R140  NaN  NaN     NaN  not a real link  139107.842252       NaN   
3033   R151  NaN  NaN     NaN  not a real link   44395.962827       NaN   
3139   R170  NaN  NaN     NaN  not a real link   50581.771138       NaN   
3325   R203  NaN  NaN     NaN  not a real link  269884.831697       NaN   
3495   R220  NaN  NaN     NaN  not a real link   17201.017465       NaN   
3563   R240  NaN  NaN     NaN  not a real link   13253.462314       NaN   
3699   R241  NaN  NaN     NaN  not a real link    3027.838464       NaN   
3749   R250  NaN  NaN     NaN  not a real link    41112.65654       NaN   
3907   R301  NaN  NaN     NaN  not a real link  226727.869434       NaN   
3949   R310  NaN  NaN     NaN  not a real link   30643.935629       NaN   
3967   R360  NaN  NaN     NaN  not a real link   91581.208219       NaN   
4225  Z1005  NaN  NaN     NaN  not a real link  282905.816228       NaN   
4331  Z1031  NaN  NaN     NaN  not a real link  166488.691329       NaN   
4407  Z1034  NaN  NaN     NaN  not a real link    18083.68593       NaN   
4471  Z1042  NaN  NaN     NaN  not a real link   95987.824597       NaN   
4549  Z1044  NaN  NaN     NaN  not a real link   28104.854858       NaN   
4675  Z1048  NaN  NaN     NaN  not a real link   27746.229994       NaN   
4781  Z1402  NaN  NaN     NaN  not a real link   65399.325126       NaN   
4899  Z2013  NaN  NaN     NaN  not a real link   184332.71387       NaN   

     roadName chainage        lat  ... model_type road_to distance_to_closest  \
1289      NaN      NaN  24.786999  ...       link     NaN                 NaN   
1473      NaN      NaN  22.489194  ...       link     NaN                 NaN   
1559      NaN      NaN   24.12275  ...       link     NaN                 NaN   
1655      NaN      NaN  23.564111  ...       link     NaN                 NaN   
2617      NaN      NaN  23.643347  ...       link     NaN                 NaN   
2685      NaN      NaN  24.308597  ...       link     NaN                 NaN   
2823      NaN      NaN  22.877542  ...       link     NaN                 NaN   
3033      NaN      NaN  22.868694  ...       link     NaN                 NaN   
3139      NaN      NaN  22.083638  ...       link     NaN                 NaN   
3325      NaN      NaN  24.802333  ...       link     NaN                 NaN   
3495      NaN      NaN  24.134806  ...       link     NaN                 NaN   
3563      NaN      NaN  24.214833  ...       link     NaN                 NaN   
3699      NaN      NaN  24.623222  ...       link     NaN                 NaN   
3749      NaN      NaN  24.867319  ...       link     NaN                 NaN   
3907      NaN      NaN  23.402597  ...       link     NaN                 NaN   
3949      NaN      NaN  24.037472  ...       link     NaN                 NaN   
3967      NaN      NaN  25.153667  ...       link     NaN                 NaN   
4225      NaN      NaN  20.554666  ...       link     NaN                 NaN   
4331      NaN      NaN  23.703486  ...       link     NaN                 NaN   
4407      NaN      NaN  23.050125  ...       link     NaN                 NaN   
4471      NaN      NaN  23.850972  ...       link     NaN                 NaN   
4549      NaN      NaN  23.405416  ...       link     NaN                 NaN   
4675      NaN      NaN  23.221485  ...       link     NaN                 NaN   
4781      NaN      NaN  23.314222  ...       link     NaN                 NaN   
4899      NaN      NaN  25.5355

In [14]:
# Drop the fake links
N1N2_filled_links.drop
N1N2_filled_links.drop(fakes.index, inplace=True)

In [15]:
# N1N2_filled_links[N1N2_filled_links['model_type']=='intersection'][['model_type','road','LRPName','road_to','closest_lrp']]

In [16]:
duplicates1 = N1N2_filled_links[N1N2_filled_links.duplicated(subset=['lat', 'lon'], keep=False)]
duplicates1

road   km            type  LRPName  \
98       N1  NaN  sideroad,right  LRP046a   
102      N1  NaN  sideroad,right  LRP047a   
244      N1  NaN  sideroad,right  LRP142a   
498      N1  NaN  sideroad,right  LRP268c   
2362     N2  NaN   sideroad,left  LRP186d   
2460     N2  NaN  sideroad,right  LRP225b   
3140   R170  NaN          Others     LRPS   
3700   R241  NaN          Others     LRPS   
4332  Z1031  NaN          Others     LRPS   
4550  Z1044  NaN          Others     LRPS   
4898  Z1402  NaN          Others     LRPE   
4900  Z2013  NaN          Others     LRPS   

                                                   name length condition  \
98                                Z1402, road to Pennai    0.0       NaN   
102                                               Z1044    0.0       NaN   
244                            Road to Karerhat (Z1031)    0.0       NaN   
498                              Road to Anowara (R170)    0.0       NaN   
2362                            Road to Raniganj (R241)    0.0       NaN   
2460                           Road to Balaganj (Z2013)    0.0       NaN   
3140  Road start from N1 (Dhaka Teknaf) road at Pati...    0.0       NaN   
3700    Road start from N2 at Soyedpur Bazar, Hobiganj.    0.0       NaN   
4332  Start from Rahman filling station at Mohhamad ...    0.0       NaN   
4550                  Start of Road From N1 at Gouripur    0.0       NaN   
4898                             Meet with N1 at Pennai    0.0       NaN   
4900            Road Start From N2 at Chandipool Sylhet    0.0       NaN   

     roadName chainage        lat  ...       model_type road_to  \
98        NaN   45.322  23.530749  ...     intersection   Z1402   
102       NaN   47.224  23.530694  ...     intersection   Z1044   
244       NaN  138.869  23.061472  ...     intersection   Z1031   
498       NaN  264.237  22.291833  ...     intersection    R170   
2362      NaN   185.77  24.613417  ...     intersection    R241   
2460      NaN  225.373  24.867305  ...     intersection   Z2013   
3140      NaN      0.0  22.291833  ...  intersourcesink      N1   
3700      NaN      0.0  24.613417  ...  intersourcesink      N2   
4332      NaN      0.0  23.061472  ...  intersourcesink      N1   
4550      NaN      0.0  23.530694  ...  intersourcesink      N1   
4898      NaN   43.849  23.530749  ...  intersourcesink      N1   
4900      NaN      0.0  24.867305  ...  intersourcesink      N2   

     distance_to_closest closest_lat closest_lon closest_lrp  \
98                   0.0   23.530749    90.78161        LRPE   
102                  0.0   23.530694   90.792444        LRPS   
244                  0.0   23.061472   91.361667        LRPS   
498                  0.0   22.291833   91.982333        LRPS   
2362                 0.0   24.613417   91.631249        LRPS   
2460                 0.0   24.867305   91.856806        LRPS   
3140                 0.0   22.291833   91.982333     LRP268c   
3700                 0.0   24.613417   91.631249     LRP186d   
4332                 0.0   23.061472   91.361667     LRP142a   
4550                 0.0   23.530694   90.792444     LRP047a   
4898                 0.0   23.530749    90.78161     LRP046a   
4900                 0.0   24.867305   91.856806     LRP225b   

                                           closest_name    closest_type  \
98                               Meet with N1 at Pennai          Others   
102                   Start of Road From N1 at Gouripur          Others   
244   Start from Rahman filling station at Mohhamad ...          Others   
498   Road start from N1 (Dhaka Teknaf) road at Pati...          Others   
2362    Road start from N2 at Soyedpur Bazar, Hobiganj.          Others   
2460            Road Start From N2 at Chandipool Sylhet          Others   
3140                             Road to Anowara (R170)  sideroad,right   
3700                            Road to Raniganj (R241)   sideroad,left   
4332                           Road to Karerhat (Z10

# Add (unique) ids for each data point, except for the intersourcesinks

In [17]:
#Change back to intersections
N1N2_filled_links['model_type'].replace('intersourcesink', 'intersection', inplace=True)

In [18]:
# First, 'correct' the lat/lon of the intersections (& intersourcesinks) by placing them on the intersecting point on the N1/N2
mask = ((N1N2_filled_links['road'] == 'N1') | 
        ((N1N2_filled_links['road'] == 'N2') & (N1N2_filled_links['road_to'] != 'N1')))

N1N2_filled_links.loc[mask & (N1N2_filled_links['model_type'] == 'intersection'), 'lat'] = \
    N1N2_filled_links.loc[mask & (N1N2_filled_links['model_type'] == 'intersection'), 'closest_lat']

N1N2_filled_links.loc[mask & (N1N2_filled_links['model_type'] == 'intersection'), 'lon'] = \
    N1N2_filled_links.loc[mask & (N1N2_filled_links['model_type'] == 'intersection'), 'closest_lon']
#         
# N1N2_filled_links[(N1N2_filled_links['model_type']=='intersection')|(N1N2_filled_links['model_type']=='intersourcesink')][['model_type','lat', 'closest_lat','lon', 'closest_lon']]

In [19]:
#For clarity rename the column "closest_distance" to "initial_distance" 
N1N2_filled_links.rename(columns={'distance_to_closest': 'initial_distance'}, inplace=True)
# N1N2_filled_links

In [20]:
N1N2_filled_links

road      km         type  LRPName  \
0        N1     NaN       Others     LRPS   
1        N1     NaN          NaN      NaN   
2        N1     1.8  Box Culvert  LRP001a   
3        N1     NaN          NaN      NaN   
4        N1   4.925  Box Culvert  LRP004b   
...     ...     ...          ...      ...   
4944  Z2013  18.732  Box Culvert  LRP018b   
4945  Z2013     NaN          NaN      NaN   
4946  Z2013  18.995  Box Culvert  LRP018c   
4947  Z2013     NaN          NaN      NaN   
4948  Z2013     NaN       Others     LRPE   

                                                   name       length  \
0     Start of Road  after Jatrabari Flyover infront...          0.0   
1                                                link 1  1776.650373   
2                                                     .         11.3   
3                                                link 2  2978.663177   
4                                                     .          6.6   
...                                                 ...          ...   
4944                                    LONGIRA CULVERT          3.0   
4945                                            link 23    251.56718   
4946                                    LONGIRA CULVERT          6.0   
4947                                                NaN  5331.997408   
4948                      Road End With Borobaga River.          0.0   

     condition                                           roadName chainage  \
0          NaN                                                NaN      0.0   
1          NaN                                                NaN      NaN   
2            A  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...      1.8   
3          NaN                                                NaN      NaN   
4            A  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...    4.925   
...        ...                                                ...      ...   
4944         A          Sylhet (Telikhal)-Sultanpur-Balaganj Road   18.732   
4945       NaN                                                NaN      NaN   
4946         A          Sylhet (Telikhal)-Sultanpur-Balaganj Road   18.995   
4947       NaN                                                NaN      NaN   
4948       NaN                                                NaN   25.045   

            lat  ...  model_type road_to initial_distance closest_lat  \
0     23.706028  ...  sourcesink     NaN              NaN         NaN   
1     23.695094  ...        link     NaN              NaN         NaN   
2     23.698739  ...      bridge     NaN              NaN         NaN   
3     23.692626  ...        link     NaN              NaN         NaN   
4     23.694664  ...      bridge     NaN              NaN         NaN   
...         ...  ...         ...     ...              ...         ...   
4944  24.706707  ...      bridge     NaN              NaN         NaN   
4945  24.703359  ...        link     NaN              NaN         NaN   
4946  24.704475  ...      bridge     NaN              NaN         NaN   
4947  24.633596  ...        link     NaN              NaN         NaN   
4948  24.657222  ...  sourcesink     NaN              NaN         NaN   

     closest_lon closest_lrp closest_name closest_type closest_chainage  \
0            NaN         NaN          NaN          NaN              NaN   
1            NaN         NaN          NaN          NaN              NaN   
2            NaN         NaN          NaN          NaN              NaN   
3            NaN         NaN          NaN          NaN              NaN   
4            NaN         NaN          NaN          NaN              NaN   
...          ...         ...          ...          ...              ...   
4944         NaN         NaN          NaN          NaN              NaN   
4945         NaN         NaN          NaN          NaN              NaN   
4946         NaN         NaN          NaN          NaN              NaN   
4947         NaN         NaN         

In [21]:
# Give a unique id to all data points
id=1
for i in N1N2_filled_links.index:
    N1N2_filled_links.at[i,'id']= id
    id+=1

# Make a dictionary of all intersections -> id mapping to coordinates
ids_dict = {}        
for i in N1N2_filled_links[N1N2_filled_links['model_type']=='intersection'].index:
    if (N1N2_filled_links.at[i,'road']=='N1') or (N1N2_filled_links.at[i,'road']=='N2'):
        ids_dict[N1N2_filled_links.at[i,'id']]= (N1N2_filled_links.at[i,'lat'],N1N2_filled_links.at[i,'lon'])   

In [22]:
## Iterate through everything and give their pairs id
ct = 0
for index, row in N1N2_filled_links.iterrows():
    lat_lon_pair = (row['lat'], row['lon'])
    for key, value_set in ids_dict.items():
        # print(type(value_set))
        if (lat_lon_pair[0]==value_set[0]) and (lat_lon_pair[1]==value_set[1]):
            ct +=1
            N1N2_filled_links.at[index, 'id'] = key
            break  # Exit the loop once a match is found

In [23]:
# Check if it worked
# ct

60

In [28]:
# Reset the index
N1N2_filled_links = N1N2_filled_links.reset_index()
N1N2_filled_links

level_0  index   road      km         type  LRPName  \
0           0      0     N1     NaN       Others     LRPS   
1           1      1     N1     NaN          NaN      NaN   
2           2      2     N1     1.8  Box Culvert  LRP001a   
3           3      3     N1     NaN          NaN      NaN   
4           4      4     N1   4.925  Box Culvert  LRP004b   
...       ...    ...    ...     ...          ...      ...   
4919     4919   4944  Z2013  18.732  Box Culvert  LRP018b   
4920     4920   4945  Z2013     NaN          NaN      NaN   
4921     4921   4946  Z2013  18.995  Box Culvert  LRP018c   
4922     4922   4947  Z2013     NaN          NaN      NaN   
4923     4923   4948  Z2013     NaN       Others     LRPE   

                                                   name       length  \
0     Start of Road  after Jatrabari Flyover infront...          0.0   
1                                                link 1  1776.650373   
2                                                     .         11.3   
3                                                link 2  2978.663177   
4                                                     .          6.6   
...                                                 ...          ...   
4919                                    LONGIRA CULVERT          3.0   
4920                                            link 23    251.56718   
4921                                    LONGIRA CULVERT          6.0   
4922                                                NaN  5331.997408   
4923                      Road End With Borobaga River.          0.0   

     condition                                           roadName  ...  \
0          NaN                                                NaN  ...   
1          NaN                                                NaN  ...   
2            A  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...  ...   
3          NaN                                                NaN  ...   
4            A  Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...  ...   
...        ...                                                ...  ...   
4919         A          Sylhet (Telikhal)-Sultanpur-Balaganj Road  ...   
4920       NaN                                                NaN  ...   
4921         A          Sylhet (Telikhal)-Sultanpur-Balaganj Road  ...   
4922       NaN                                                NaN  ...   
4923       NaN                                                NaN  ...   

     road_to initial_distance closest_lat closest_lon closest_lrp  \
0        NaN              NaN         NaN         NaN         NaN   
1        NaN              NaN         NaN         NaN         NaN   
2        NaN              NaN         NaN         NaN         NaN   
3        NaN              NaN         NaN         NaN         NaN   
4        NaN              NaN         NaN         NaN         NaN   
...      ...              ...         ...         ...         ...   
4919     NaN              NaN         NaN         NaN         NaN   
4920     NaN              NaN         NaN         NaN         NaN   
4921     NaN              NaN         NaN         NaN         NaN   
4922     NaN              NaN         NaN         NaN         NaN   
4923     NaN              NaN         NaN         NaN         NaN   

     closest_name closest_type closest_chainage intersection      id  
0             NaN          NaN              NaN          NaN     1.0  
1             NaN          NaN              NaN          NaN     2.0  
2             NaN          NaN              NaN          NaN     3.0  
3             NaN          NaN              NaN          NaN     4.0  
4             NaN          NaN              NaN          NaN     5.0  
...           ...          ...              ...          ...     ...  
4919          NaN          NaN              NaN          NaN  4920.0  
4920          NaN          NaN              NaN          NaN  4921.0  
4921          NaN          NaN              NaN    

In [25]:
# N1N2_filled_links.id.value_counts().value_counts()

count
1    4864
2      30
Name: count, dtype: int64

In [26]:
# N1N2_filled_links.model_type.value_counts()

model_type
link            2449
bridge          2385
intersection      60
sourcesink        30
Name: count, dtype: int64

In [27]:
N1N2_filled_links.to_csv('../data/semi-processed/cleaned_data_final.csv')